Logistic Regression

In [1]:
from src.logistic_regression import *

### Importing HuggingFace

In [2]:
train_dataset = load_dataset('imdb', split='train')
test_dataset = load_dataset('imdb', split='test')

Reusing dataset imdb (/home/surenis/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)
Reusing dataset imdb (/home/surenis/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


In [3]:
x_train, y_train = generate_x_and_y(train_dataset)
x_test, y_test = generate_x_and_y(test_dataset)

### Logistic Regression

Let's search for all the words finishing with !

As it would make no sense that the ! character appears before the end of the word

it appears no words ends with ! in the training set

Thus, there is no need to check for ! in the documents

In [ ]:
X_train_features = LoRegression(x_train, y_train)

  9%|▉         | 2280/25000 [01:26<13:59, 27.06it/s]

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train_features, y_train)

In [ ]:
#create X_test using loRegression to have a usable informations
X_test_features = LoRegression(x_test, y_test)

In [ ]:
y_pred = clf.predict(X_test_features)
print(X_test_features.shape, y_pred.shape)

In [ ]:
clf.score(X_test_features, y_test)

In [ ]:
from sklearn.metrics import classification_report

target_names = ['negative review', 'positive review']
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
acc = ""
for word in x_test[101]:
    acc = acc + word + " "
print(acc)

In [ ]:
print("Review was labelled as: ",y_pred[101],"instead of: ",y_test[101])

In [ ]:
X_test_features[101]